# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from glob import glob

import pyopia.classify
import pyopia.process
import pyopia.statistics
import pyopia.io

# List raw data files

In [2]:
raw_data = "raw_data"
raw_files = glob(os.path.join(raw_data, '*.silc'))
raw_files

['raw_data\\D20181101T142731.838206.silc',
 'raw_data\\D20181101T142732.772669.silc',
 'raw_data\\D20181101T142733.765097.silc',
 'raw_data\\D20181101T142734.292793.silc']

# Define custom functions or non-defult settings

In [3]:
from functools import partial

extractparticles_function = partial(pyopia.process.extract_particles,
                                    export_outputpath='test_export',
                                    min_length=0)

statextract_function = partial(pyopia.process.statextract,
                           minimum_area=12, threshold=0.98, real_time_stats=False,
                           max_coverage=30, max_particles=5000,
                           extractparticles_function=extractparticles_function)

# Initialise the CNN

In [4]:
# initialise the Classify class
cl = pyopia.classify.Classify()

# at this point you could overwrite custom classification functions,
# like cl.proc_predict() to work with a different classification network

# load the model (this example can be found here:
#    https://github.com/SINTEF/PySilCam/wiki/ml_models/keras_model.zip)
cl.load_model(model_path="C:\model\keras_model.h5")

# Calculate particle statistics for each image

In [5]:
for filename in raw_files:
    
    # load a silcam image
    img = np.load(filename, allow_pickle=False)
    
    # get the timestamp of the image (in this case from the filename)
    timestamp = pd.to_datetime(os.path.splitext(os.path.basename(filename))[0][1:])

    # setup the 'data' tuple with an image number, timestamp and the image
    data = (1, timestamp, img)

    # call process_image
    stats = pyopia.process.process_image(cl, data, statextract_function=statextract_function)
    
    pyopia.io.write_stats('proc/test.h5', stats)
    break

Processing time stamp 2018-11-01 14:27:31.838206
segment
clean
2.2% saturation
measure
  121 particles found


c:\users\emlynd\documents\github\pyopia\pyopia\process.py:230: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  meta.attrs['Modified'] = str(pd.datetime.now())


EXTRACTING 121 IMAGES from 121
  Image 1 processed in 1.98 sec (0.5 Hz). 


ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

## The STATS DataFrame

In [ ]:
# print the stats DataFrame
stats

# Analysis, statistics and plotting

In [ ]:
# there are lots of functions in pyopia.statistics. Here is just an example.

# Calculate the volume distribution from the stats DataFrame.
# (Usually several images would be needed for statistics to converge.
# This can be done by appending new image stats to the DataFrame)
dias, vd = pyopia.statistics.vd_from_stats(stats, 24)

# plot the volume distribution
plt.plot(dias, vd)
plt.xscale('log')
plt.xlabel('ECD [um]')
plt.ylabel('Volume Distribution [uL/sample vol.]')